# PydanticOutputParser 가이드

## 개요

**PydanticOutputParser**는 언어 모델의 자유 형식 텍스트 출력을 구조화된 데이터 객체로 변환하는 도구입니다. 이 클래스를 사용하면 AI 모델의 응답을 일관된 형식으로 파싱하고 검증할 수 있습니다.

## PydanticOutputParser의 핵심 구성 요소

### 필수 메서드

| 메서드 | 기능 | 역할 |
|--------|------|------|
| **get_format_instructions()** | 형식 지침 생성 | AI 모델에게 원하는 출력 형식을 지시 |
| **parse()** | 텍스트 파싱 | 문자열을 Pydantic 모델 객체로 변환 |

### 작동 원리

PydanticOutputParser는 다음과 같은 워크플로우로 동작합니다:

1. **형식 지침 제공**: `get_format_instructions()`로 AI에게 JSON 스키마 기반 출력 형식 전달
2. **AI 응답 생성**: 언어 모델이 지침에 따라 구조화된 텍스트 생성  
3. **데이터 변환**: `parse()` 메서드로 텍스트를 Pydantic 객체로 변환
4. **타입 검증**: Pydantic을 통한 데이터 타입 및 형식 검증

## 기술적 요구사항

### 필수 패키지

- **langchain-core**: OutputParser 기본 클래스
- **pydantic**: 데이터 모델 정의 및 검증
- **langchain-openai**: 언어 모델 인터페이스

### Pydantic 모델 설계 원칙

- **Field 설명**: 각 필드에 명확한 `description` 제공
- **타입 안전성**: 적절한 Python 타입 힌트 사용
- **검증 규칙**: 필요시 Pydantic 검증 규칙 적용

## 활용 사례

- **이메일 파싱**: 발신자, 제목, 내용 등 구조화된 정보 추출
- **문서 요약**: 주요 내용을 정해진 형식으로 요약
- **데이터 추출**: 비정형 텍스트에서 특정 정보 추출

## 참고 자료

- [Pydantic 공식 문서](https://docs.pydantic.dev/latest/)
- [LangChain OutputParsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

In [ ]:
# 실시간 출력을 위한 import (스트리밍 응답을 보기 좋게 출력)
from langchain_teddynote.messages import stream_response

In [ ]:
# 필요한 라이브러리 임포트
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

---

## 실습 데이터 준비

### 시나리오 설정

비즈니스 협력 제안 이메일을 분석하여 다음 정보를 자동으로 추출하는 시스템을 구현합니다.

#### 추출 대상 정보

| 항목 | 설명 |
|------|------|
| **발신자 정보** | 이름 및 이메일 주소 |
| **이메일 제목** | 메일 제목 전체 |
| **내용 요약** | 메일 본문의 핵심 내용 |
| **미팅 일정** | 본문에 언급된 날짜 및 시간 |

### 실무 활용 가능성

이러한 자동화 시스템은 다음과 같은 상황에서 유용합니다:

- **대량 이메일 처리**: 일일 수십~수백 건의 비즈니스 이메일 자동 분류
- **CRM 시스템 연동**: 추출된 정보를 고객 관리 시스템에 자동 입력
- **업무 효율성 향상**: 수동 분류 작업 시간 단축

In [ ]:
# 실습용 이메일 데이터 (비즈니스 협력 제안 메일 예시)
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

---

## 파서 사용 전후 비교

### 기본 AI 응답의 한계

PydanticOutputParser를 사용하지 않은 경우의 문제점을 확인해봅니다.

#### 구조화되지 않은 응답의 특징

- **자연어 형태**: 사람이 읽기 쉽지만 프로그램 처리가 어려움
- **일관성 부족**: 매번 다른 형식으로 응답 생성
- **데이터 추출 어려움**: 특정 정보를 프로그래밍적으로 접근하기 복잡

### 실험 목적

동일한 이메일 데이터에 대해 다음 두 방식의 결과를 비교합니다:

1. **기본 프롬프트**: 파서 없이 자연어 응답
2. **구조화된 파싱**: PydanticOutputParser 적용

In [ ]:
# 출력 파서 없이 기본 체인 구성
from langchain_core.prompts import PromptTemplate

# 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\\n\\n{email_conversation}"
)

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 체인 구성 (프롬프트 + 모델)
chain = prompt | llm

# 체인 실행 (스트리밍 방식)
answer = chain.stream({"email_conversation": email_conversation})

# 실시간 출력 및 결과 반환
output = stream_response(answer, return_output=True)

In [ ]:
# 파서 없이 실행한 결과 출력 (구조화되지 않은 텍스트 형태)
print(output)

### 결과 분석

파서 없이 실행된 결과는 자연어 형태로 제공되어 사람이 읽기에는 적합하지만, 프로그램이 데이터를 구조적으로 처리하기에는 부적합합니다.

---

## Pydantic 모델 설계

### EmailSummary 모델 구조

이메일 정보를 체계적으로 구조화하기 위한 데이터 모델을 정의합니다.

#### 모델 설계 원칙

| 구성 요소 | 목적 | 중요성 |
|----------|------|--------|
| **Field 타입** | 데이터 타입 명시 | 타입 안전성 보장 |
| **Description** | AI 지침 제공 | 정확한 정보 추출을 위한 가이드 |
| **Validation** | 데이터 검증 | 입력값의 유효성 검사 |

#### Field Description 작성 지침

`Field` 의 `description` 매개변수는 AI 모델에게 전달되는 명령어 역할을 수행합니다:

- **명확성**: 추출해야 할 정보의 범위를 구체적으로 명시
- **일관성**: 동일한 유형의 정보는 항상 같은 방식으로 추출
- **정확성**: 원하는 데이터 형식과 내용을 정확히 설명

#### 모델 활용 시 고려사항

- **확장성**: 필요에 따라 필드 추가/수정 용이
- **재사용성**: 유사한 이메일 처리 작업에서 재활용 가능
- **유지보수**: 비즈니스 요구사항 변경 시 모델 수정을 통한 대응

In [ ]:
# 이메일 정보를 구조화하기 위한 Pydantic 모델 정의
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 인스턴스 생성 (EmailSummary 모델 기반)
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [ ]:
# AI 모델에게 전달할 형식 지침 확인 (JSON 스키마 형태로 생성됨)
print(parser.get_format_instructions())

### Format Instructions 분석

`get_format_instructions()` 메서드가 생성하는 지침은 AI 모델에게 전달되는 상세한 작업 명세서입니다.

#### 생성되는 지침의 구성 요소

| 항목 | 내용 | 목적 |
|------|------|------|
| **JSON 스키마** | 출력해야 할 데이터 구조 명시 | 정확한 형식 지정 |
| **필드 설명** | 각 필드별 추출해야 할 정보 안내 | 내용 정확성 향상 |
| **형식 규칙** | JSON 문법 준수 사항 | 파싱 오류 방지 |

---

## 프롬프트 템플릿 구성

### 템플릿 설계 원칙

효과적인 구조화된 출력을 위한 프롬프트 템플릿 구성 요소입니다.

#### 주요 구성 요소

| 요소 | 역할 | 필수 여부 |
|------|------|-----------|
| **question** | 사용자의 구체적 요청 | 필수 |
| **email_conversation** | 분석 대상 텍스트 | 필수 |
| **format** | 출력 형식 지침 | 필수 |

#### partial() 함수의 활용

`partial()` 함수는 템플릿의 일부 변수를 미리 설정하는 기능입니다:

**사용 전**:
```python
{question}, {email_conversation}, {format}
```

**partial 적용 후**:
```python
{question}, {email_conversation}, [format 지침 자동 삽입]
```

이를 통해 실행 시점에 question과 email_conversation만 제공하면 됩니다.

In [ ]:
# 구조화된 출력을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    """You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# 프롬프트에 파서의 형식 지침을 부분적으로 적용 (미리 format 필드를 채워놓음)
prompt = prompt.partial(format=parser.get_format_instructions())

In [ ]:
print(parser.get_format_instructions())

---

## 체인 구성

### LangChain 파이프라인 구조

LangChain의 파이프라인 연산자(`|`)를 사용하여 처리 단계를 연결합니다.

#### 기본 체인 구조

```python
chain = prompt | llm
```

#### 처리 흐름

| 단계 | 입력 | 출력 | 역할 |
|------|------|------|------|
| **1. 프롬프트** | 사용자 입력 + 템플릿 | 완성된 프롬프트 | 지시사항 생성 |
| **2. LLM** | 완성된 프롬프트 | AI 응답 텍스트 | 내용 생성 |

### 체인 설계의 장점

#### 모듈성
- **독립적 구성 요소**: 각 단계를 별도로 수정/교체 가능
- **재사용성**: 동일한 체인을 여러 작업에 활용
- **테스트 용이성**: 각 단계별로 독립적 테스트 가능

#### 확장성  
- **단계 추가**: 필요시 추가 처리 단계 삽입
- **병렬 처리**: 여러 체인을 동시 실행
- **조건부 실행**: 상황에 따른 다른 처리 경로 설정

In [ ]:
# 프롬프트와 모델을 연결한 체인 생성 (파서는 별도로 적용 예정)
chain = prompt | llm

---

## 체인 실행 및 결과 분석

### 실행 결과의 특성

체인 실행 후 얻는 결과는 여전히 텍스트 형태입니다.

#### 중간 결과물의 특징

- **JSON 유사 형태**: AI가 지시사항에 따라 구조화 시도
- **문자열 타입**: 프로그래밍적 접근을 위해서는 추가 파싱 필요
- **형식 일관성**: 동일한 구조의 출력 생성

### 다음 단계

이 텍스트 결과를 실제 Python 객체로 변환하기 위해 PydanticOutputParser의 `parse()` 메서드를 사용합니다.

In [ ]:
# 체인 실행 (스트리밍 방식으로 응답 생성)
response = chain.stream(
    {
        "email_conversation": email_conversation,  # 이메일 내용 입력
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",  # 사용자 질문
    }
)

# 스트리밍 응답을 실시간으로 출력하고 결과 저장
output = stream_response(response, return_output=True)

---

## 텍스트에서 구조화된 데이터로 변환

### parse() 메서드의 역할

PydanticOutputParser의 핵심 기능인 텍스트 파싱 과정을 분석합니다.

#### 변환 과정

| 단계 | 작업 내용 | 기술적 구현 |
|------|----------|-------------|
| **1. 텍스트 분석** | JSON 형태 문자열 식별 | 정규식 또는 파싱 라이브러리 |
| **2. JSON 파싱** | 문자열을 Python 딕셔너리로 변환 | json.loads() 또는 유사 기능 |
| **3. 객체 생성** | 딕셔너리를 Pydantic 모델로 변환 | Pydantic 검증 엔진 |
| **4. 타입 검증** | 필드별 타입 및 제약조건 확인 | Pydantic 검증 규칙 |

#### 변환 후 데이터 접근

구조화된 객체로 변환된 후에는 다음과 같은 방식으로 데이터에 접근할 수 있습니다:

```python
result.person      # 발신자 이름
result.email       # 이메일 주소  
result.subject     # 메일 제목
result.summary     # 내용 요약
result.date        # 미팅 일정
```

### 파싱의 장점

- **타입 안전성**: 각 필드가 정의된 타입으로 보장
- **자동 완성**: IDE에서 속성 자동 완성 지원
- **오류 방지**: 잘못된 필드 접근 시 컴파일 타임 오류 발생

In [ ]:
# 텍스트 응답을 EmailSummary 객체로 변환 (구조화된 데이터로 파싱)
structured_output = parser.parse(output)
print(structured_output)

In [ ]:
structured_output.date

---

## 완전한 파이프라인 구성

### 통합 체인 구조

모든 처리 과정을 하나의 파이프라인으로 연결합니다.

#### 3단계 파이프라인

```python
chain = prompt | llm | parser
```

#### 완전 자동화 처리 흐름

| 단계 | 구성 요소 | 입력 | 출력 |
|------|----------|------|------|
| **1** | PromptTemplate | 사용자 데이터 | 완성된 프롬프트 |
| **2** | ChatOpenAI | 완성된 프롬프트 | JSON 형태 텍스트 |
| **3** | PydanticOutputParser | JSON 텍스트 | EmailSummary 객체 |

### 통합 파이프라인의 이점

#### 효율성
- **단일 호출**: 한 번의 `invoke()` 또는 `stream()` 호출로 전체 처리 완료
- **자동 연결**: 각 단계 간 데이터 전달이 자동으로 처리
- **오류 처리**: 파이프라인 차원에서 통합된 오류 관리

#### 유지보수성
- **모듈식 구조**: 각 구성 요소를 독립적으로 수정 가능
- **테스트 용이성**: 전체 파이프라인 또는 개별 구성 요소별 테스트
- **재사용성**: 다른 프로젝트나 유사한 작업에서 활용 가능

### 실행 결과 특성

최종 결과는 바로 사용 가능한 EmailSummary 객체로, 추가적인 파싱 과정 없이 프로그램에서 직접 활용할 수 있습니다.

In [ ]:
# 파서까지 포함한 완전한 체인 구성 (프롬프트 + 모델 + 파서)
chain = prompt | llm | parser

In [ ]:
# 완전한 체인 실행 (한 번에 구조화된 결과 생성)
response = chain.invoke(
    {
        "email_conversation": email_conversation,  # 이메일 내용 입력
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",  # 사용자 질문
    }
)

# 결과가 자동으로 EmailSummary 객체 형태로 반환됨
response

---

## 대안 접근법: with_structured_output()

PydanticOutputParser보다 간단한 구조화된 출력 방법을 제공합니다.

### with_structured_output() 방식의 특징

이 방법은 별도의 프롬프트 템플릿이나 파서 설정 없이 직접 구조화된 출력을 얻을 수 있습니다.

#### 두 방식 비교

| 특성 | PydanticOutputParser | with_structured_output() |
|------|---------------------|--------------------------|
| **설정 복잡도** | 높음 | 낮음 |
| **프롬프트 제어** | 완전 제어 | 제한적 |
| **커스터마이징** | 높은 자유도 | 기본 설정 위주 |
| **실행 속도** | 보통 | 빠름 |
| **학습 곡선** | 가파름 | 완만함 |

#### 사용 권장 상황

**with_structured_output() 권장:**
- 빠른 프로토타입 개발
- 간단한 구조화 작업
- 최소한의 설정으로 결과 확인

**PydanticOutputParser 권장:**
- 복잡한 프롬프트 엔지니어링 필요
- 세밀한 출력 형식 제어 필요
- 프로덕션 환경에서의 안정적 처리

### 기능적 제약사항

`with_structured_output()` 방식은 스트리밍 기능을 지원하지 않습니다. 실시간 출력이 필요한 경우에는 PydanticOutputParser를 사용해야 합니다.

In [ ]:
# 이메일 정보를 구조화하기 위한 Pydantic 모델 정의
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [ ]:
# with_structured_output을 사용한 구조화된 출력
llm_with_structered = ChatOpenAI(
    temperature=0.1,
    model_name="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
).with_structured_output(EmailSummary)

In [ ]:
# with_structured_output을 사용한 간편한 구조화된 출력
answer = llm_with_structered.invoke(email_conversation)
answer

In [ ]:
# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

### 참고사항

**중요**: `with_structured_output()` 메서드는 `stream()` 기능을 지원하지 않습니다. 실시간 스트리밍이 필요한 애플리케이션에서는 PydanticOutputParser를 사용하시기 바랍니다.